In [1]:
import warnings
# 忽视警告
warnings.filterwarnings('ignore')

import cv2
from PIL import Image
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader
from model import Backbone

# -------------------------- 请加载您最满意的模型 ---------------------------
# 加载模型(请加载你认为的最佳模型)
# 加载模型,加载请注意 model_path 是相对路径, 与当前文件同级。
# 如果你的模型是在 results 文件夹下的 star.h5 模型，则 model_path = 'results/star.h5'
model_path = 'res50.pth'

# 加载模型，如果采用keras框架训练模型，则 model=load_model(model_path)
class ArcfaceNet(nn.Module):
    def __init__(self, classes=10):
        super(ArcfaceNet, self).__init__()
        self.Arcnet = Backbone(num_layers=50,drop_ratio=0.6,mode='ir_se')
        #self.Arcnet.load_state_dict(torch.load('model_ir_se50.pth'))
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = self.Arcnet(x)
        out = self.fc(x)
        return out
model = ArcfaceNet(classes=10)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)
model.eval()
# ---------------------------------------------------------------------------

def predict(img):
    """
    加载模型和模型预测
    主要步骤:
        1.图片处理
        2.用加载的模型预测图片的类别
    :param img: 经 cv2.imread(file_path) 读取后的图片
    :return: string, 模型识别图片的类别, 
            共 'CL', 'FBB', 'HG', 'HJ', 'LHR', 'LSS', 'LYF', 'PYY', 'TY', 'YM' 10 个类别
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    labels = {0:'CL',1:'FBB',2:'HG',3:'HJ',4:'LHR',5:'LSS',6:'LYF',7:'PYY',8:'TY',9:'YM'}
    test_trans = T.Compose([
            T.Resize((112, 112)),
            T.ToTensor(),  # 转化为张量
            T.Normalize([0], [1]),  # 归一化
        ])
    img=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img = test_trans(img)
    img = img.view(1,img.shape[0],img.shape[1],img.shape[2])
    model.eval()
    with torch.no_grad():
        result = model(img)
        result = labels[np.argmax(result.numpy())]
    # 获取输入图片的类别
    y_predict = result

    # -------------------------------------------------------------------------
    
    # 返回图片的类别
    return y_predict

In [2]:
predict(cv2.imread('gbb.jpg'))

'FBB'

In [ ]:
state_dict = torch.load('new.pth', map_location="cpu")
torch.save(state_dict, 'old3.pth', _use_new_zipfile_serialization=False)